In [ ]:
import fxcmpy
import socketio
import numpy as np
import pandas as pd
from pandas_datareader import data as wb
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sympy import *
init_printing()
import xlrd
import itertools
from datetime import datetime as dt
import pytz
from itertools import combinations
from itertools import product
from itertools import groupby
from astropy.table import QTable, Table, Column
from astropy import units as u
from IPython.display import HTML
import requests
import json
from scipy.signal import argrelextrema

TOKEN = 'b199bb5a15fa26957990a3fb823a51bd473cca55'
con = fxcmpy.fxcmpy(access_token=TOKEN, log_level='error', server='demo', log_file='log.txt')

In [ ]:
##  INITIAL SETUP

## WATCHLIST = ['AUD/CAD','AUD/CHF','AUD/JPY','AUD/NZD','AUD/USD','CAD/CHF','CAD/JPY','CHF/JPY','EUR/AUD','EUR/CAD','EUR/CHF','EUR/GBP','EUR/JPY','EUR/NZD','EUR/USD','GBP/AUD','GBP/CAD','GBP/CHF','GBP/JPY','GBP/USD','NZD/CAD','NZD/CHF','NZD/JPY','NZD/USD','USD/CAD','USD/CHF','USD/JPY']

pair = ['EUR/JPY']
h = len(pair)


with pd.ExcelWriter('C:/Users/joeca/Desktop/Finance/FOREX/Hercules_Prototype_Mark_I/MarkI_TESTInstTrend.xlsx') as writerdtt:

    for x in pair:

## TO BEGIN, WE WILL RUN A SHORT PROGRAM TO FIGURE OUT HOW TO SET THE TIMEFRAME FOR THE LEVELS CALCULATION


        Q = con.get_candles(x, period='H4', number=120)      ## START WITH 120 CANDLESTICKS (20 DAYS) FOR CURRENT MIDPOINT
        Q['open'] = (Q['bidopen']+Q['askopen'])/2
        Q['close'] = (Q['bidclose']+Q['askclose'])/2
        Q['high'] = (Q['bidhigh']+Q['askhigh'])/2
        Q['low'] = (Q['bidlow']+Q['asklow'])/2

        D = Q
        D = D.tz_localize('Etc/GMT-4')
        D = D.tz_convert('UTC')
        D = D.assign(MOVE=(D['close'] - D['open']))
        AvgCandle = np.average(abs(D['MOVE']))
        l = int(D['close'].count())
        lin = np.linspace(1,l,num=l)

        thh = 12                                         ## CANDLESTICK INTERVAL FOR NET TREND (SET TO 12 CANDLES NOW)
        tl = l - thh
        tlin = np.linspace(1,tl,num=tl)
        tlinb = np.linspace(1,thh,num=thh)
        TREND = []
        for t in tlinb:
            TREND.append(0)
        i = thh                                                                                       
        for t in tlin:
            ii = i - thh
            movetrend = np.sum(list(D.iloc[ii:i,13]))
            if movetrend > 0:
                mtrend = 1
            elif movetrend < 0:
                mtrend = -1
            else:
                mtrend = 0
            i = i + 1
            TREND.append(mtrend)
        D['TREND'] = TREND
